In [2]:

#create an empty structure model.import warnings

from causalnex.structure import StructureModel
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings("ignore")  # silence warnings

sm = StructureModel()

In [4]:
# import the dataset
df = pd.read_csv('/home/martin/Desktop/10-Academy/week-8/Causal-Inference/data/merged_result.csv')
df.head(5)

,id,order_id,driver_id,driver_action,lat,lng,Trip_Origin,Trip_Destination,Weekday,Trip_Start_Day,Holiday,Rain,Distance,Speed
0,1,392001,243828,accepted,6.602207,3.270465,"6.6010417,3.2766339","6.4501069,3.3916154",3.0,1.0,False,Rain,21.048262,0.026943
1,2,392001,243588,rejected,6.592097,3.287445,"6.6010417,3.2766339","6.4501069,3.3916154",3.0,1.0,False,Rain,21.048262,0.026943
2,3,392001,243830,rejected,6.596133,3.281784,"6.6010417,3.2766339","6.4501069,3.3916154",3.0,1.0,False,Rain,21.048262,0.026943
3,4,392001,243539,rejected,6.596142,3.280526,"6.6010417,3.2766339","6.4501069,3.3916154",3.0,1.0,False,Rain,21.048262,0.026943
4,5,392001,171653,rejected,6.609232,3.288800,"6.6010417,3.2766339","6.4501069,3.3916154",3.0,1.0,False,Rain,21.048262,0.026943


In [6]:
# choose non-numeric columns
import numpy as np

struct_data = df.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

['driver_action', 'Trip_Origin', 'Trip_Destination', 'Holiday', 'Rain']


In [10]:
drop_col = ['Trip_Origin','Trip_Destination','Trip_Start_Day','order_id','lat','lng','id', 'driver_id']
data = df.drop(columns=drop_col)
data.head(5)

,driver_action,Weekday,Holiday,Rain,Distance,Speed
0,accepted,3.0,False,Rain,21.048262,0.026943
1,rejected,3.0,False,Rain,21.048262,0.026943
2,rejected,3.0,False,Rain,21.048262,0.026943
3,rejected,3.0,False,Rain,21.048262,0.026943
4,rejected,3.0,False,Rain,21.048262,0.026943


In [12]:
# Replace 'rejected' with 0 and 'accepted' with 1 in the 'driver_action' column
data['driver_action'] = data['driver_action'].replace({'rejected': 0, 'accepted': 1})
data['Holiday'] = data['Holiday'].replace({'false': 0, 'true': 1})
data['Rain'] = data['Rain'].replace({'No Rain': 0, 'Rain': 1})



In [13]:
# Save the modified DataFrame to a new CSV file
data.to_csv("causal.csv", index=False)

In [3]:
data = pd.read_csv('/home/martin/Desktop/10-Academy/week-8/Causal-Inference/causal.csv')
data.head(5)

,driver_action,Weekday,Holiday,Rain,Distance,Speed
0,1,3.0,False,1,21.048262,0.026943
1,0,3.0,False,1,21.048262,0.026943
2,0,3.0,False,1,21.048262,0.026943
3,0,3.0,False,1,21.048262,0.026943
4,0,3.0,False,1,21.048262,0.026943


In [49]:
 #  use label encoding non-numeric variables to make data numeric
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.head(5)

,driver_action,weekday,holiday,rain,distance,speed
0,1,1.0,0.0,0.0,13.039051,216.316059
1,0,1.0,0.0,0.0,13.039051,216.316059
2,0,1.0,0.0,0.0,13.039051,216.316059
3,0,1.0,0.0,0.0,13.039051,216.316059
4,0,1.0,0.0,0.0,13.039051,216.316059


In [50]:
struct_data["weekday"].value_counts()


1.0    1355842
0.0     199876
Name: weekday, dtype: int64

In [51]:
struct_data.head()

,driver_action,weekday,holiday,rain,distance,speed
0,1,1.0,0.0,0.0,13.039051,216.316059
1,0,1.0,0.0,0.0,13.039051,216.316059
2,0,1.0,0.0,0.0,13.039051,216.316059
3,0,1.0,0.0,0.0,13.039051,216.316059
4,0,1.0,0.0,0.0,13.039051,216.316059


In [52]:
import pandas as pd
import numpy as np

# Impute NaN with column means
for col in struct_data.columns:
    if struct_data[col].isnull().any():
        mean_value = struct_data[col].mean()
        struct_data[col].fillna(mean_value, inplace=True)

In [53]:
#apply the NOTEARS algorithm to learn the structure.

from causalnex.structure.notears import from_pandas
sm = from_pandas(struct_data)

In [ ]:
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("../screenshots/03_fully_connected.html")

In [ ]:
sm.remove_edges_below_threshold(0.5)
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("../screenshots/03_thresholded.html")